# 第7章 因子分析及Python应用

In [ ]:
#%cd "E:/应统电子书讲义/多元python"

In [ ]:
#输出初始化设置
import pandas as pd                        #加载数据分析包
pd.set_option('display.precision',4)       #设置数据框输出精度
pd.set_option('display.max_rows',14)       #显示数据框最大行数
from pandas import DataFrame as DF         #设置数据框别名

## 7.1 因子分析模型

### 7.1.1 因子概念的提出

### 7.1.2 因子分析的用途

### 7.1.3 因子模型的提出

In [ ]:
d71=pd.read_excel('mvsData.xlsx','d71',index_col=0)
print(d71)

In [ ]:
d71.corr()                              #计算相关阵

## 7.2 因子载荷及解释

In [ ]:
#!pip install factor_analyzer

### 7.2.1 主因子估计法

In [ ]:
#def FA_disp(fa,Indexs):  #显示因子信息
#    Columns=['F'+str(i) for i in range(1,fa.n_factors+1)]    
#    return pd.DataFrame(fa.loadings_,Indexs,Columns)      

In [ ]:
#显示因子名称
def Factors(fa):return ['F'+str(i) for i in range(1,fa.n_factors+1)]    

In [ ]:
from factor_analyzer import FactorAnalyzer as FA
Fp=FA(n_factors=6,method='principal',rotation=None).fit(d71.values) #拟合6个主因子
DF(Fp.loadings_,d71.columns,Factors(Fp)) #显示因子载荷

In [ ]:
Fp1=FA(n_factors=3,method='principal',rotation=None).fit(d71.values) #取前3个主因子
Fp1_load=DF(Fp1.loadings_,d71.columns,Factors(Fp1))  #定义因子载荷阵
Fp1_load

### 7.2.2 极大似然估计法

In [ ]:
Fm=FA(n_factors=6,method='ml',rotation=None).fit(d71.values) #取前6个主因子
DF(Fm.loadings_,d71.columns,Factors(Fm)) 

In [ ]:
Fm1=FA(n_factors=3,method='ml',rotation=None).fit(d71.values)
Fm1_load=DF(Fm1.loadings_,d71.columns,Factors(Fm1)) 
Fm1_load

### 7.2.3 因子载荷的意义

In [ ]:
Vars=['方差','贡献率','累计贡献率']
Fp1_Vars=DF(Fp1.get_factor_variance(),Vars,Factors(Fp1))
Fp1_Vars #主因子法方差贡献 

In [ ]:
Fm1_Vars=DF(Fm1.get_factor_variance(),Vars,Factors(Fm1))
Fm1_Vars #极大似然法方差贡献 

In [ ]:
#主因子法共同度，uniquenesses（不一致性）
Fp1_load['共同度']=1-Fp1.get_uniquenesses()
print(Fp1_load)

In [ ]:
#极大似然法共同度，uniquenesses（不一致性）
Fm1_load['共同度']=1-Fm1.get_uniquenesses()
print(Fm1_load)

## 7.3 因子旋转

### 7.3.1 因子旋转方法

### 7.3.2 如何进行旋转

In [ ]:
Fp2=FA(3,method='principal',rotation='varimax').fit(d71.values) #varimax旋转
Fp2_Vars=DF(Fp2.get_factor_variance(),Vars,Factors(Fp2))
Fp2_Vars

In [ ]:
Fp2_load=DF(Fp2.loadings_,d71.columns,Factors(Fp2))  
Fp2_load  #旋转前因子载荷

In [ ]:
Fp2_load['共同度']=1-Fp2.get_uniquenesses()
print(Fp2_load)  #旋转后因子载荷及共同度

## 7.4 因子得分

### 7.4.1 因子得分的计算

In [ ]:
#pd.set_option('display.max_rows',15) #设置最大显示行数
Fp1_scores=DF(Fp1.transform(d71.values),d71.index,Factors(Fp1))
print(Fp1_scores)   #旋转前因子得分

In [ ]:
#旋转后因子得分
Fp2_scores=DF(Fp2.transform(d71.values),d71.index,Factors(Fp2))
print(Fp2_scores)

### 7.4.2 因子得分信息图

In [ ]:
import matplotlib.pyplot as plt            
plt.rcParams['font.sans-serif']=['SimHei'];  #中文黑体SimHei
plt.rcParams['axes.unicode_minus']=False; #正常显示图中正负号
def Scoreplot(Scores): #自定得分图绘制函数
    plt.plot(Scores.iloc[:,0],Scores.iloc[:,1],'*'); 
    plt.xlabel(Scores.columns[0]);plt.ylabel(Scores.columns[1])
    plt.axhline(y=0,ls=':');plt.axvline(x=0,ls=':')
    for i in range(len(Scores)):
        plt.text(Scores.iloc[i,0],Scores.iloc[i,1],Scores.index[i])

In [ ]:
#自定得分图绘制函数，见第6章6.3.2节
Scoreplot(Fp2_scores[['F1','F2']]) 

In [ ]:
Scoreplot(Fp2_scores[['F1','F3']])

In [ ]:
Scoreplot(Fp2_scores[['F2','F3']])

In [ ]:
def Biplot(Load,Score): #双向因子信息重叠图
    Scoreplot(Score)
    for i in range(Load.shape[0]):
        plt.arrow(0,0,Load.iloc[i,0],Load.iloc[i,1],color='r')
        plt.text(Load.iloc[i,0],Load.iloc[i,1],Load.index[i],color='b')

In [ ]:
Biplot(Fp2_load,Fp2_scores)

### 7.4.3 综合得分及排名

In [ ]:
def FArank(Vars,Scores): #计算综合因子得分与排名
    Vi=Vars.values[0]
    Wi=Vi/sum(Vi);Wi
    Fi=Scores.dot(Wi)
    Ri=Fi.rank(ascending=False).astype(int);
    return(pd.DataFrame({'因子得分':Fi,'因子排名':Ri}))

In [ ]:
print(FArank(Fp1_Vars,Fp1_scores))

In [ ]:
print(FArank(Fp2_Vars,Fp2_scores))

## 7.5 因子分析步骤

### 7.5.1 因子分析的准备工作

（1）简单相关分析

In [ ]:
d31=pd.read_excel('mvsData.xlsx','d31',index_col=0)
print(d31.corr())                  #计算相关阵

（2）KMO统计量

In [ ]:
import factor_analyzer as fa
kmo=fa.calculate_kmo(d31) #计算KMO
print('KMO: %5.4f'%kmo[1])

（3）Bartlett's球状检验

In [ ]:
chisq=fa.calculate_bartlett_sphericity(d31) #进行bartlett检验
print('卡方值 = %8.4f, p值 = %5.4f'%(chisq[0],chisq[1]))

### 7.5.2 因子分析的过程

In [ ]:
def FAscores(X,m=2,rot='varimax'): #因子分析综合评价函数
    import factor_analyzer as fa
    kmo=fa.calculate_kmo(X) 
    chisq=fa.calculate_bartlett_sphericity(X) #进行bartlett检验
    print('KMO检验: KMO值=%6.4f 卡方值=%8.4f, p值=%5.4f'%(kmo[1],chisq[0],chisq[1]))
    from factor_analyzer import FactorAnalyzer as FA
    Fp=FA(n_factors=m,method='principal',rotation=rot).fit(X.values)
    vars=Fp.get_factor_variance()
    Factor=['F%d' %(i+1) for i in range(m)]
    Vars=pd.DataFrame(vars,['方差','贡献率','累计贡献率'],Factor)
    print("\n方差贡献:\n",Vars)
    Load=pd.DataFrame(Fp.loadings_,X.columns,Factor) 
    Load['共同度']=1-Fp.get_uniquenesses()
    print("\n因子载荷:\n",Load)
    Scores=pd.DataFrame(Fp.transform(X.values),X.index,Factor)    
    print("\n因子得分:\n",Scores)
    Vi=vars[0]
    Wi=Vi/sum(Vi);Wi
    Fi=Scores.dot(Wi)
    Ri=Fi.rank(ascending=False).astype(int);
    print("\n综合排名:\n")
    return pd.DataFrame({'综合得分':Fi,'综合排名':Ri})

In [ ]:
pd.set_option('display.max_rows',31) 
FAscores(d31,m=2,rot='varimax')

## 7.6 实际中进行因子分析

### 7.6.1 基本分析

### 7.6.2 扩展分析

## 案例7 上市公司经营业绩评价的因子分析

In [ ]:
Case7=pd.read_excel('mvscase.xlsx','Case7',index_col=0); #Case7

In [ ]:
print(FAscores(Case7,m=4))